In [1]:
# Import libraries
import os, shutil, sys, time, re, requests, csv, datetime, pytz
import yaml
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the OOI M2M tool
sys.path.append("/home/andrew/Documents/OOI-CGSN/ooinet/ooinet/")
from m2m import M2M

In [3]:
# Import user info for connecting to OOINet via M2M
userinfo = yaml.load(open("../../../../QAQC_Sandbox/user_info.yaml"), Loader=yaml.FullLoader)
username = userinfo["apiname"]
token = userinfo["apikey"]

In [4]:
OOINet = M2M(username, token)
Dev01 = M2M(username, token)

In [5]:
# Reset the M2M location to ooinet-dev1-west.intra.oceanobservatories.org
for key in Dev01.URLS:
    url = Dev01.URLS.get(key)
    if "opendap" in url:
        dev1_url = re.sub("opendap", "opendap-dev1-west.intra", url)
    else:
        dev1_url = re.sub("ooinet","ooinet-dev1-west.intra", url)
    Dev01.URLS[key] = dev1_url

In [6]:
#OOINet.search_datasets(array=instrument="PHSEN")

### Production (OOINet) Data

In [6]:
refdes = "GI03FLMA-RIM01-02-CTDMOG040"

In [7]:
datastreams = OOINet.get_datastreams(refdes)
datastreams

,refdes,method,stream
0,GI03FLMA-RIM01-02-CTDMOG040,recovered_host,ctdmo_ghqr_offset_recovered
1,GI03FLMA-RIM01-02-CTDMOG040,recovered_host,ctdmo_ghqr_sio_mule_instrument
2,GI03FLMA-RIM01-02-CTDMOG040,recovered_inst,ctdmo_ghqr_instrument_recovered
3,GI03FLMA-RIM01-02-CTDMOG040,telemetered,ctdmo_ghqr_sio_mule_instrument
4,GI03FLMA-RIM01-02-CTDMOG040,telemetered,ctdmo_ghqr_sio_offset


In [8]:
metadata = OOINet.get_metadata(refdes)
for m in metadata["particleKey"]:
    if "ph" in m:
        print(m)

In [9]:
deployments = OOINet.get_deployments(refdes)
deployments

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,8,CGINS-CTDMOG-10223,1887,59.77095,-39.8836,30.0,2021-08-13 13:22:00,NaT,AR60-01,None
0,7,CGINS-CTDMOG-13593,3447,59.767117,-39.843617,30.0,2020-08-19 14:48:00,2021-08-17 09:36:00,AR46,AR60-01
0,6,CGINS-CTDMOG-11643,2552,59.7701,-39.88419,30.0,2019-08-07 13:34:00,2020-08-27 10:54:00,AR35-05,AR46
0,5,CGINS-CTDMOG-13560,3767,59.76805,-39.84252,30.0,2018-06-10 13:59:00,2019-08-11 08:25:00,AR30-03,AR35-05
0,4,CGINS-CTDMOG-13438,3041,59.7717,-39.8827,30.0,2017-08-08 15:13:00,2018-06-15 07:59:00,AR21,AR30-03
0,3,CGINS-CTDMOG-11612,2434,59.7677,-39.84123,30.0,2016-07-12 15:59:00,2017-08-12 10:00:00,AR7-01,AR21
0,2,CGINS-CTDMOG-13419,3097,59.77067,-39.8801,31.0,2015-08-18 23:47:00,2016-07-17 07:55:00,AT30-01,AR7-01
0,1,CGINS-CTDMOG-11613,2501,59.76683,-39.8425,26.0,2014-09-12 20:11:00,2015-08-18 09:00:00,KN221-04,AT30-01


In [10]:
method = "recovered_inst"
stream = "ctdmo_ghqr_instrument_recovered"

In [11]:
def clean_catalog(catalog, stream, deployments):
    """Clean up the netCDF catalog of unwanted datasets"""
    # Parse the netCDF datasets to only get those with the datastream in its name
    datasets = []
    for dset in catalog:
        check = dset.split("/")[-1]
        if stream in check:
            datasets.append(dset)
        else:
            pass
    
    # Next, check that the netCDF datasets are not empty by getting the timestamps in the
    # datasets and checking if they are 
    catalog = datasets
    datasets = []
    for dset in catalog:
        # Get the timestamps
        timestamps = dset.split("_")[-1].replace(".nc","").split("-")
        t1, t2 = timestamps
        # Check if the timestamps are equal
        if t1 == t2:
            pass
        else:
            datasets.append(dset)
            
    # Next, determine if the dataset is either for the given instrument
    # or an ancillary instrument which supplies and input variable
    catalog = datasets
    datasets = []
    ancillary = []
    for dset in catalog:
        if re.search(stream, dset.split("/")[-1]) is None:
            ancillary.append(dset)
        else:
            datasets.append(dset)
            
    # Finally, check that deployment numbers match what is in deployments metadata
    catalog = datasets
    datasets = []
    for dset in catalog:
        dep = re.findall("deployment[\d]{4}", dset)[0]
        depNum = int(dep[-4:])
        if depNum not in list(deployments["deploymentNumber"]):
            pass
        else:
            datasets.append(dset)
            
    return datasets

In [13]:
thredds_url = OOINet.get_thredds_url(refdes, method, stream)
#thredds_url = "https://opendap-west.oceanobservatories.org/thredds/catalog/ooi/areed@whoi.edu/20220329T172343104Z-CP03ISSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html"

Waiting for GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered to process.
Waiting: 100%|████████████████████████████████| 400/400 [01:03<00:00,  6.32it/s]


In [14]:
# Access the catalog
catalog = OOINet.get_thredds_catalog(thredds_url)
catalog

['catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0001_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20140912T201501-20150818T084501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0002_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20150819T000001-20160717T074501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0003_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20160712T160001-20170812T094501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0004_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_in

In [15]:
# Clean up the catalog to eliminate "empty" datasets
catalog = clean_catalog(catalog, stream, deployments)
catalog

['catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0001_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20140912T201501-20150818T084501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0002_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20150819T000001-20160717T074501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0003_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20160712T160001-20170812T094501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0004_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_in

In [16]:
catalog = [x for x in catalog if "blank" not in x]
catalog

['catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0001_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20140912T201501-20150818T084501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0002_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20150819T000001-20160717T074501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0003_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20160712T160001-20170812T094501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153247548Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0004_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_in

In [17]:
OOINet.REFDES = refdes

In [18]:
production_data = OOINet.load_netCDF_datasets(catalog)


Loading netCDF_files for GI03FLMA-RIM01-02-CTDMOG040:
[########################################] | 100% Completed | 29.1s


In [19]:
production_data

<xarray.Dataset>
Dimensions:                                      (time: 242505)
Coordinates:
  * time                                         (time) datetime64[ns] 2014-0...
Data variables: (12/33)
    practical_salinity                           (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_executed  (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_results   (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_executed           (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_results            (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctd_time                                     (time) datetime64[ns] dask.array<chunksize=(32595,), meta=np.ndarray>
    ...                                           ...
    ctdmo_seawater_pressure                      (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qc_executed               (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    depth                                        (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    inductive_id                                 (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    density_qc_results                           (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_pressure_qartod_executed      (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
Attributes: (12/70)
    node:                               RIM01
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdmo_ghqr_instrument_recovered
    ...                                 ...
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                59.76683
    lon:                                -39.8425
    Location_name:                      Global Irminger Sea Flanking Subsurfa...

In [ ]:
#water_files = [x for x in catalog if "water_recovered" in x.split("/")[-1]]
#air_files = [x for x in catalog if "air_recovered" in x.split("/")[-1]]
#water_files

### Dev01 Data

In [20]:
dev01_thredds_url = Dev01.get_thredds_url(refdes, method, stream)
#dev01_thredds_url = 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/catalog/ooi/areed@whoi.edu/20220329T173040530Z-CP03ISSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html'

Waiting for GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered to process.
Waiting: 100%|████████████████████████████████| 400/400 [01:03<00:00,  6.34it/s]


In [21]:
dev01_catalog = Dev01.get_thredds_catalog(dev01_thredds_url)
dev01_catalog

['catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0001_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20140912T201501-20150818T084501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0002_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20150819T000001-20160717T074501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0003_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20160712T160001-20170812T094501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0004_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_in

In [22]:
dev01_catalog = clean_catalog(dev01_catalog, stream, deployments)
dev01_catalog

['catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0001_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20140912T201501-20150818T084501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0002_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20150819T000001-20160717T074501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0003_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered_20160712T160001-20170812T094501.nc',
 'catalog.html?dataset=ooi/areed@whoi.edu/20220331T153446751Z-GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_instrument_recovered/deployment0004_GI03FLMA-RIM01-02-CTDMOG040-recovered_inst-ctdmo_ghqr_in

In [23]:
dev01_catalog = [x for x in dev01_catalog if "blank" not in x]

In [24]:
Dev01.REFDES = refdes

In [25]:
dev01_data = Dev01.load_netCDF_datasets(dev01_catalog)
dev01_data


Loading netCDF_files for GI03FLMA-RIM01-02-CTDMOG040:
[########################################] | 100% Completed | 28.0s


<xarray.Dataset>
Dimensions:                                      (time: 242505)
Coordinates:
  * time                                         (time) datetime64[ns] 2014-0...
Data variables: (12/33)
    practical_salinity                           (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_executed  (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_results   (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_executed           (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_results            (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctd_time                                     (time) datetime64[ns] dask.array<chunksize=(32595,), meta=np.ndarray>
    ...                                           ...
    ctdmo_seawater_pressure                      (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qc_executed               (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    depth                                        (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    inductive_id                                 (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    density_qc_results                           (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_pressure_qartod_executed      (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
Attributes: (12/70)
    node:                               RIM01
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdmo_ghqr_instrument_recovered
    ...                                 ...
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                59.76683
    lon:                                -39.8425
    Location_name:                      Global Irminger Sea Flanking Subsurfa...

### Load netCDF files from local directory

In [ ]:
save_dir = f"/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/QARTOD/Testing/data/testing/{refdes}/"

In [ ]:
netCDF_files = [save_dir+f for f in os.listdir(save_dir)]
#water_files = [x for x in netCDF_files if "water_recovered" in x.split("/")[-1]]
#air_files = [x for x in netCDF_files if "air_recovered" in x.split("/")[-1]]

In [ ]:
netCDF_files

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
OOINet.REFDES = refdes

In [ ]:
# -------------------------------
# Third, check and remove any files which are malformed
# and remove the bad ones
netCDF_files = OOINet._check_files(netCDF_files)

# Load the datasets into a concatenated xarray DataSet
with ProgressBar():
    print("\n"+f"Loading netCDF_files for {OOINet.REFDES}:")
    ds = xr.open_mfdataset(netCDF_files, preprocess=OOINet._preprocess, parallel=True)

# Add in the English name of the dataset
refdes = "-".join(ds.attrs["id"].split("-")[:4])
vocab = OOINet.get_vocab(refdes)
ds.attrs["Location_name"] = " ".join((vocab["tocL1"].iloc[0],
                                      vocab["tocL2"].iloc[0],
                                      vocab["tocL3"].iloc[0]))

In [ ]:
import gc
gc.collect()

In [ ]:
ds

## QARTOD Comparison

In [ ]:
param = "partial_pressure_co2_ssw"
data_variables = []
for var in ds.variables:
    if param in var and "qc" not in var:
        print(var)
        data_variables.append(var)

In [ ]:
# First, cut down the dataset size to be more managable
ds = ds[data_variables]

#

In [ ]:
#del ds
gc.collect()

### Production vs Dev01
First, check that Dev01 datasets matched released production datasets flagging

In [26]:
param = "practical_salinity"

In [27]:
# Cut the production data down to the size of the dev01 data
tmin = dev01_data["time"].min()
tmax = dev01_data["time"].max()
production_data = production_data.sel(time=slice(tmin, tmax))
dev01_data = dev01_data.sel(time=slice(tmin, tmax))

In [28]:
comparison = (production_data[f"{param}_qartod_results"] == dev01_data[f"{param}_qartod_results"])
comparison

<xarray.DataArray 'practical_salinity_qartod_results' (time: 242505)>
dask.array<eq, shape=(242505,), dtype=bool, chunksize=(40607,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2014-09-12T20:15:01 ... 2021-08-13T13:15:01

In [29]:
(~comparison).sum().compute()

<xarray.DataArray 'practical_salinity_qartod_results' ()>
array(0)

In [30]:
value_check = (production_data[param] == dev01_data[param])
value_check

<xarray.DataArray 'practical_salinity' (time: 242505)>
dask.array<eq, shape=(242505,), dtype=bool, chunksize=(40607,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2014-09-12T20:15:01 ... 2021-08-13T13:15:01

In [31]:
production_data[f"{param}_qartod_results"][~comparison]

<xarray.DataArray 'practical_salinity_qartod_results' (time: 0)>
dask.array<getitem, shape=(0,), dtype=float32, chunksize=(0,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 
Attributes:
    flag_values:    [1 2 3 4 9]
    flag_meanings:  pass not_evaluated suspect_or_of_high_interest fail missi...
    standard_name:  sea_water_practical_salinity status_flag
    long_name:      Practical Salinity QARTOD Summary Flag
    references:     https://ioos.noaa.gov/project/qartod https://github.com/i...
    comment:        Summary QARTOD test flags. For each datum, the flag is se...
    coordinates:    time lat lon depth
    _ChunkSizes:    10000

In [32]:
dev01_data[f"{param}_qartod_results"][~comparison]

<xarray.DataArray 'practical_salinity_qartod_results' (time: 0)>
dask.array<getitem, shape=(0,), dtype=float32, chunksize=(0,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 
Attributes:
    flag_values:    [1 2 3 4 9]
    flag_meanings:  pass not_evaluated suspect_or_of_high_interest fail missi...
    standard_name:  sea_water_practical_salinity status_flag
    long_name:      Practical Salinity QARTOD Summary Flag
    references:     https://ioos.noaa.gov/project/qartod https://github.com/i...
    comment:        Summary QARTOD test flags. For each datum, the flag is se...
    coordinates:    time lat lon depth
    _ChunkSizes:    10000

### QARTOD values
Next, load the QARTOD tables from github and parse them into dictionaries.

Changes: None

In [33]:
inst = "CTDMO"
#param = "ctdbp_seawater_temperature"

In [34]:
import io
import json
def loadQARTOD(refDes,param,sensorType):
    
    (site,node,sensor1,sensor2) = refDes.split('-')
    sensor = sensor1 + '-' + sensor2
    
    ### Load climatology and gross range values
    githubBaseURL = 'https://raw.githubusercontent.com/oceanobservatories/qc-lookup/master/qartod/'
    if 'ph_seawater' in param:
        ClimParam = 'seawater_ph'
    else:
        ClimParam = param
    clim_URL = githubBaseURL + sensorType + '/climatology_tables/' + refDes + '-' + ClimParam + '.csv'
    grossRange_URL = githubBaseURL + sensorType + '/' + sensorType + '_qartod_gross_range_test_values.csv'
    download = requests.get(grossRange_URL)
    if download.status_code == 200:
        df_grossRange = pd.read_csv(io.StringIO(download.content.decode('utf-8')))
        paramString = "{'inp': '" + param + "'}"
        qcConfig = df_grossRange.qcConfig[(df_grossRange.subsite == site) 
                                          & (df_grossRange.node == node) 
                                          & (df_grossRange.sensor == sensor) 
                                          & (df_grossRange.parameters == paramString)]
        qcConfig_json = qcConfig.values[0].replace("'", "\"")
        grossRange_dict = json.loads(qcConfig_json)
    else:
        print('error retriving gross range data')
        grossRange_dict = {}

    download = requests.get(clim_URL)
    if download.status_code == 200:
        df_clim = pd.read_csv(io.StringIO(download.content.decode('utf-8')))
        climRename = {
                'Unnamed: 0':'depth',
                '[1, 1]':'1',
                '[2, 2]':'2',
                '[3, 3]':'3',
                '[4, 4]':'4',
                '[5, 5]':'5',
                '[6, 6]':'6',
                '[7, 7]':'7',
                '[8, 8]':'8',
                '[9, 9]':'9',
                '[10, 10]':'10',
                '[11, 11]':'11',
                '[12, 12]':'12'           
            } 
        
        df_clim.rename(columns=climRename, inplace=True)
        clim_dict = df_clim.set_index('depth').to_dict()
    else:
        print('error retriving climatology data')
        clim_dict = {}
    
    return(grossRange_dict,clim_dict)

In [36]:
grossRange_dict, clim_dict = loadQARTOD(refdes, param, inst.lower())
grossRange_dict, clim_dict

({'qartod': {'gross_range_test': {'suspect_span': [34.59, 35.15],
    'fail_span': [0, 42]}}},
 {'1': {'[0, 0]': '[34.7208, 35.0941]'},
  '2': {'[0, 0]': '[34.7592, 35.0678]'},
  '3': {'[0, 0]': '[34.7611, 35.0562]'},
  '4': {'[0, 0]': '[34.7235, 35.0742]'},
  '5': {'[0, 0]': '[34.6771, 35.0948]'},
  '6': {'[0, 0]': '[34.0327, 35.7029]'},
  '7': {'[0, 0]': '[33.9754, 35.714]'},
  '8': {'[0, 0]': '[33.6146, 36.0327]'},
  '9': {'[0, 0]': '[33.9233, 35.7081]'},
  '10': {'[0, 0]': '[34.5312, 35.1244]'},
  '11': {'[0, 0]': '[34.6218, 35.0896]'},
  '12': {'[0, 0]': '[34.6572, 35.116]'}})

### Add Climatology Values
Next, add the climatology min and max values to the dataset as new data variables, based on the month of the data.

Changes:
* Renamed "climatologyMin/climatologyMax" to "{parameter name}\_climatologyMin/climatologyMax" in order to allow multiple parameter climatologies to be stored in an given dataset
* Preallocated the climatology arrays with nans instead of zeros to skip the later step of backfilling nans.
* Utilize dask to get the months (as integers) in the time variable of the dataset. This avoids loading the data into memory.
* Utilize direct assignment of the climatologyMin/Max values for each month on the dataset variable arrays. This again keeps the dataset out-of-memory.

In [37]:
import dask.array as da
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import ast

def add_climatology_values(ds, param, clim_dict):
    """Adds climatology mins and maxes to the dataset timeseries
    
    Parameters
    ----------
    ds: xarray.Dataset
        Dataset to add climatology values to, with primary dimension "time"
    param: str
        Name of parameter in the passed xarray.Dataset which to add
        climatology values to
    clim_dict: dict
        A dictionary of the climatology values for the given dataset
        loaded from the qartod gitHub repo
        
    Returns
    -------
    ds: xarray.Dataset
        An xarray dataset with climatology mins and maxes added for the given
        parameter (param) to the dataset
        
    Note: Will need to add a pressure function to make this match the original functionality
    """
    
    # First, create a variable name to store the data
    varNameMin = f"{param}_climatologyMin"
    varNameMax = f"{param}_climatologyMax"
    
    # Next, pre-allocate an array with the data
    ds[varNameMin] = ds[param].astype(float) * np.nan
    ds[varNameMax] = ds[param].astype(float) * np.nan
    
    # Get the months
    time = da.from_array(ds.time.dt.month)
    months = np.unique(time).compute()
    
    # Add the climatology min and max based on the month of the measurement
    for month in months:
        climatology = ast.literal_eval(clim_dict[str(month)][str([0, 0])])
        ds[varNameMin][(ds.time.dt.month == month)] = climatology[0]
        ds[varNameMax][(ds.time.dt.month == month)] = climatology[1]
        
    return ds

In [38]:
import dask.array as da
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import ast

def build_climatology_array(ds, clim_dict, press_param, param_name, platform):
    """Adds climatology mins and maxes to the dataset timeseries
    
    Parameters
    ----------
    ds: xarray.Dataset
        Dataset to add climatology values to, with primary dimension "time"
    param: str
        Name of parameter in the passed xarray.Dataset which to add
        climatology values to
    press_param: str
        Name of the pressure parameter for profilers and other vehicles with
        climatology values that are pressure dependent
    clim_dict: dict
        A dictionary of the climatology values for the given dataset
        loaded from the qartod gitHub repo
        
    Returns
    -------
    ds: xarray.Dataset
        An xarray dataset with climatology mins and maxes added for the given
        parameter (param) to the dataset
        
    Note: Will need to add a pressure function to make this match the original functionality
    """
    ds['climatologyMin'] = ds[param_name].astype('float') * np.nan
    ds['climatologyMax'] = ds[param_name].astype('float') * np.nan
    
    # Get the months
    time = da.from_array(ds.time.dt.month)
    months = np.unique(time).compute()
    
    # Iterate through the months
    # This is the slow part - it takes 12*num_press_brackets*O(NlogN) time
    for month in months:
        # First, check if need to filter again by pressure
        if platform == "profiler":
            # Get the pressure dictionary for the given month
            pres_dict = clim_dict.get(str(month))
            for pressure_range in pres_dict.keys():
                # Parse the pressure range
                p = re.search(r'\[(.+),(.+)\]', pressure_range)
                pmin, pmax = float(p.group(1)), float(p.group(2))
                # Parse the climatology
                climatology = pres_dict.get(pressure_range)
                c = re.search(r'\[(.+),(.+)\]', climatology)
                cmin, cmax = float(c.group(1)), float(c.group(2))
                # Now assign the climatology min/max
                ds["climatologyMin"][(ds.time.dt.month == month) &
                                     (ds[press_param] >= pmin) & 
                                     (ds[press_param] <= pmax)] = cmin
                ds["climatologyMax"][(ds.time.dt.month == month) &
                                     (ds[press_param] >= pmin)  &
                                     (ds[press_param] <= pmax)] = cmax
        elif platform == "fixed":
            climatology = ast.literal_eval(clim_dict[str(month)][str([0, 0])])
            cmin, cmax = climatology[0], climatology[1]
            ds["climatologyMin"][(ds.time.dt.month == month)] = cmin
            ds["climatologyMax"][(ds.time.dt.month == month)] = cmax
        else:
            pass
        
    return ds

In [39]:
press_param = None
platform = "fixed"

In [40]:
toy_data = production_data
toy_data["practical_salinity_qartod_results"][0:10] = 3

In [41]:
production_data = build_climatology_array(production_data, clim_dict, press_param, param, platform)
dev01_data = build_climatology_array(dev01_data, clim_dict, press_param, param, platform)
toy_data = build_climatology_array(toy_data, clim_dict, press_param, param, platform)

### Add QARTOD flags
Next, want to calculate the QARTOD flags for the gross range and climatology values and add them to the dataset. 
Changes:
* Renamed the "gr_flag/clim_flag" to "{parameter name}\_gr_flag/\_clim_flag" in order to allow multiple parameters to be tested in a single dataset.
* Utilize direct assignment of the QARTOD flags to avoid loading data into memory.

In [42]:
def create_QARTOD_flags(ds, param, grossRange):
    """Function to add the gross range and climatology flags"""
    
    # Add the gross range flags for a param
    gr_flag = f"{param}_gr_flag"
    ds[gr_flag] = ds[param].astype("int64") * 0 + 1
    gr_suspect = grossRange["qartod"]["gross_range_test"]["suspect_span"]
    gr_fail = grossRange["qartod"]["gross_range_test"]["fail_span"]
    ds[gr_flag][(ds[param] < gr_suspect[0]) | (ds[param] > gr_suspect[1])] = 3
    ds[gr_flag][(ds[param] < gr_fail[0]) | (ds[param] > gr_fail[1])] = 4
     
    # Climatology flags
    clim_flag = f"{param}_clim_flag"
    ds[clim_flag] = ds[param].astype("int64") * 0 + 1
    ds[clim_flag][(ds["climatologyMin"].isnull()) | (ds["climatologyMax"].isnull())] = 2
    ds[clim_flag][(ds[param] < ds["climatologyMin"]) | (ds[param] > ds["climatologyMax"])] = 3
    
    # Check for not evaluated locations
    not_eval = ds[param].isnull()
    ds[gr_flag][not_eval] = 9
    ds[clim_flag][not_eval] = 9
    
    return ds

In [43]:
production_data = create_QARTOD_flags(production_data, param, grossRange_dict)
dev01_data = create_QARTOD_flags(dev01_data, param, grossRange_dict)
toy_data = create_QARTOD_flags(toy_data, param, grossRange_dict)

### Compare test values
Now, want to compare the values calculated locally with the values returned by OOINet in the "qartod_executed" variables.

Changes:
* Don't iterate through each data point
* Change the data type of the {parameter name}\_qartod_executed data array to string to be interperable
* With the type changed to string, can use the xarray built-in string methods (.str) to parse each value in the "qartod_executed" array
* Changed the name of "qartod_gr/qartod_clim" to "{parameter name}\_qartod_gr/\_qartod_clim" to allow multiple parameters to be stored in the same dataset
* Run the test comparison and store as "{parameter name}\_gr_comparison/\_clim_comparison" as a boolean array. This will allow us to quickly count the comparison (using sum) and mask the parameter being tested.

In [44]:
def run_comparison(ds, param):
    
    # First, identify the test order of the qartod tests run
    qartod_name = f"{param}_qartod_executed"
    test_order = ds[qartod_name].attrs["tests_executed"].strip("'").replace(" ", "").split(",")
    
    # Second, identify the index of each test
    clim_index = test_order.index("climatology_test")
    gr_index = test_order.index("gross_range_test")
    
    # Next, convert the OOINet-run QARTOD flags to interperable strings
    ds[qartod_name] = ds[qartod_name].astype(str)
    
    # Parse the qartod flags into the separate test flags
    ds[f"{param}_qartod_gr"] = ds[qartod_name].str.get(gr_index).astype("int")
    ds[f"{param}_qartod_clim"] = ds[qartod_name].str.get(clim_index).astype("int")
    
    # Compare the OOI Qartod with local Qartod
    ds[f"{param}_gr_comparison"] = ds[f"{param}_qartod_gr"] != ds[f"{param}_gr_flag"]
    ds[f"{param}_clim_comparison"] = ds[f"{param}_qartod_clim"] != ds[f"{param}_clim_flag"]
    
    return ds

In [45]:
toy_data[f"{param}_qartod_executed"][0:10] = '33'

In [46]:
production_data = run_comparison(production_data, param)
dev01_data = run_comparison(dev01_data, param)
toy_data = run_comparison(toy_data, param)

In [96]:
for x in production_data.practical_salinity_qartod_executed:
    if x.values != '11' | x.values:
        print(x.time.values)
        print(x.values)

In [102]:
dev01_data.practical_salinity_qartod_executed.load()

<xarray.DataArray 'practical_salinity_qartod_executed' (time: 242505)>
array(['33', '11', '11', ..., '11', '11', '11'], dtype='<U64')
Coordinates:
  * time     (time) datetime64[ns] 2014-09-12T20:15:01 ... 2021-08-13T13:15:01
Attributes:
    comment:         Individual QARTOD test flags. For each datum, flags are ...
    tests_executed:  gross_range_test, climatology_test
    standard_name:   sea_water_practical_salinity status_flag
    long_name:       Practical Salinity Individual QARTOD Flags
    references:      https://ioos.noaa.gov/project/qartod https://github.com/...
    coordinates:     time lat lon depth
    _ChunkSizes:     510

In [103]:
np.unique(dev01_data.practical_salinity_qartod_executed)

array(['11', '13', '31', '33',
       'Bad HeapObject.dataSize=id=16, refCount=0, dataSize=452117892851',
       'Bad HeapObject.dataSize=id=16, refCount=0, dataSize=533495497202'],
      dtype='<U64')

In [101]:
production_data.practical_salinity_qartod_executed.where((production_data.practical_salinity_qartod_executed == "Bad HeapObject.dataSize=id=16, refCount=0, dataSize=452117892851") |
                                                        (production_data.practical_salinity_qartod_executed == "Bad HeapObject.dataSize=id=16, refCount=0, dataSize=533495497202"), drop=True)

<xarray.DataArray 'practical_salinity_qartod_executed' (time: 2)>
array(['Bad HeapObject.dataSize=id=16, refCount=0, dataSize=452117892851',
       'Bad HeapObject.dataSize=id=16, refCount=0, dataSize=533495497202'],
      dtype=object)
Coordinates:
  * time     (time) datetime64[ns] 2016-07-02T19:15:01 2017-07-21T11:15:01
Attributes:
    comment:         Individual QARTOD test flags. For each datum, flags are ...
    tests_executed:  gross_range_test, climatology_test
    standard_name:   sea_water_practical_salinity status_flag
    long_name:       Practical Salinity Individual QARTOD Flags
    references:      https://ioos.noaa.gov/project/qartod https://github.com/...
    coordinates:     time lat lon depth
    _ChunkSizes:     510

In [68]:
production_data

<xarray.Dataset>
Dimensions:                                      (time: 242505)
Coordinates:
  * time                                         (time) datetime64[ns] 2014-0...
Data variables: (12/41)
    practical_salinity                           (time) float64 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_executed  (time) |S64 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctdmo_seawater_conductivity_qartod_results   (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_executed           (time) <U64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_results            (time) float32 dask.array<chunksize=(32595,), meta=np.ndarray>
    ctd_time                                     (time) datetime64[ns] dask.array<chunksize=(32595,), meta=np.ndarray>
    ...                                           ...
    practical_salinity_gr_flag                   (time) int64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_clim_flag                 (time) int64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_gr                 (time) int64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_qartod_clim               (time) int64 dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_gr_comparison             (time) bool dask.array<chunksize=(32595,), meta=np.ndarray>
    practical_salinity_clim_comparison           (time) bool dask.array<chunksize=(32595,), meta=np.ndarray>
Attributes: (12/70)
    node:                               RIM01
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdmo_ghqr_instrument_recovered
    ...                                 ...
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                59.76683
    lon:                                -39.8425
    Location_name:                      Global Irminger Sea Flanking Subsurfa...

In [65]:
x.values.dtype == '<U2'

True

### Execute the comparison
So far, all the work we've done hasn't actually run any processing. Everything has been done as a set of dask instructions to execute when we call compute().

Below, I first just count the number of missed flags by summing the comparison results, since each "missed" flag is stored as a boolean ```True```, which ```.sum()``` counts as a 1. 

In [47]:
from dask.diagnostics import ProgressBar

In [48]:
with ProgressBar():
    for var in production_data.variables:
        if "comparison" in var:
            result = production_data[var].sum().compute()
            print(f"Missed flags for {var}: {result.values}")

[##########                              ] | 27% Completed |  3.5s


ValueError: invalid literal for int() with base 10: 'B'

In [49]:
with ProgressBar():
    for var in dev01_data.variables:
        if "comparison" in var:
            result = dev01_data[var].sum().compute()
            print(f"Missed flags for {var}: {result.values}")

[###############                         ] | 39% Completed |  2.8s


ValueError: invalid literal for int() with base 10: 'B'

In [72]:
with ProgressBar():
    for var in toy_data.variables:
        if "comparison" in var:
            result = toy_data[var].sum().compute()
            print(f"Missed flags for {var}: {result.values}")

[########################################] | 100% Completed | 11.4s
Missed flags for ctdbp_seawater_temperature_gr_comparison: 0
[########################################] | 100% Completed | 11.7s
Missed flags for ctdbp_seawater_temperature_clim_comparison: 0
[########################################] | 100% Completed | 10.4s
Missed flags for practical_salinity_gr_comparison: 10
[########################################] | 100% Completed | 10.3s
Missed flags for practical_salinity_clim_comparison: 10


### Plot some data with bad data flagged

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

ax[0].plot(production_data.time, production_data[param], linestyle="", marker=".", color="tab:blue")
ax[0].plot(production_data.where((production_data[f"{param}_qartod_clim"] == 3))["time"],
           production_data.where((production_data[f"{param}_qartod_clim"] == 3))[param],
           color="tab:red", marker=".", linestyle="")
ax[0].grid()
ax[0].set_ylim()

ax[1].plot(production_data.time, production_data[param], linestyle="", marker=".", color="tab:blue")
ax[1].plot(production_data.where(production_data[f"{param}_clim_flag"] == 3)["time"],
           production_data.where(production_data[f"{param}_clim_flag"] == 3)[param],
           color="tab:red", marker=".", linestyle="")
ax[1].grid()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

ax[0].plot(dev01_data.time, dev01_data[param], linestyle="", marker=".", color="tab:blue")
ax[0].plot(dev01_data.where((dev01_data[f"{param}_qartod_clim"] == 3))["time"],
           dev01_data.where((dev01_data[f"{param}_qartod_clim"] == 3))[param],
           color="tab:red", marker=".", linestyle="")
ax[0].grid()

ax[1].plot(dev01_data.time, dev01_data[param], linestyle="", marker=".", color="tab:blue")
ax[1].plot(dev01_data.where(dev01_data[f"{param}_clim_flag"] == 3)["time"],
           dev01_data.where(dev01_data[f"{param}_clim_flag"] == 3)[param],
           color="tab:red", marker=".", linestyle="")
ax[1].grid()

### To Do
Need to add in pressure bracket handling so that I can do profilers (although I don't have any profilers for CGSN up on Dev1). 

Also need to add in function to print out the time-stamp of when qartod flags are mis-flaged.

In [ ]:
def pressureBracket(pressure,clim_dict):
    bracketList = []
    pressBracket = 'notFound'

    for bracket in clim_dict['1'].keys():
        x = re.search(r'\[(.+),(.+)\]', bracket)
        if x:
            bracketList.append([int(x.group(1)),int(x.group(2))])
        else:
            print('bracket parsing error for ' + bracket)
    for bracket in bracketList:
        if (pressure >= bracket[0]) & (pressure < bracket[1]):
            pressBracket = bracket
            break
    
    return pressBracket